#### Importing necessary libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


#### Importing the basic cleaned data
Property prices of Mumbai City source:https://www.99acres.com/property-rates-and-price-trends-in-mumbai

In [8]:
pp_1=pd.read_csv("https://raw.githubusercontent.com/dibyendutapadar/mumbai_city_analysis/master/mumbai_property_price.csv")
pp_1.head()

,Locality Name,buy_rate,buy_rate_trim,buy_rate_lower,buy_rate_higher,buy_rate_avg,Buy Rates Q/Q,Rent_1B,Rent_1B_trim,Rent_1B_Lower,Rent_1B_Upper,Rent_1B_avg,Rent_1B_persqf,Rent_2B,Rent_2B_trim,Rent_2B_Lower,Rent_2B_Upper,Rent_2B_avg,Rent_2B_persqf,Rent_3B,Rent_3B_trim,Rent_3B_Lower,Rent_3B_Upper,Rent_3B_avg,Rent_3B_persqf
0,4 Bunglows,"Rs.18,828 - 20,825/sq. ft.","18,828 - 20,825",18828.0,20825.0,19826.5,-1.48%,"Rs.24,225 - 31,025","24,225 - 31,025",24225.0,31025.0,27625.0,139%,"Rs.43,350 - 48,450","43,350 - 48,450",43350.0,48450.0,45900.0,232%,"Rs.53,780 - 59,755","53,780 - 59,755",53780.0,59755.0,56767.5,286%
1,Aarey Milk Colony,"Rs.7,225 - 7,862/sq. ft.","7,225 - 7,862",7225.0,7862.0,7543.5,4.71%,"Rs.16,371 - 20,464","16,371 - 20,464",16371.0,20464.0,18417.5,244%,"Rs.21,216 - 23,338","21,216 - 23,338",21216.0,23338.0,22277.0,295%,"Rs.23,899 - 29,330","23,899 - 29,330",23899.0,29330.0,26614.5,353%
2,Abhyudaya Nagar,"Rs.26,605 - 28,858/sq. ft.","26,605 - 28,858",26605.0,28858.0,27731.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Rs.63,954 - 68,395","63,954 - 68,395",63954.0,68395.0,66174.5,239%,NaN,NaN,NaN,NaN,NaN,NaN
3,Adaigaon,"Rs.4,462 - 4,802/sq. ft.","4,462 - 4,802",4462.0,4802.0,4632.0,7.92%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Adharwadi,"Rs.4,675 - 5,398/sq. ft.","4,675 - 5,398",4675.0,5398.0,5036.5,-7.26%,"Rs.5,142 - 6,078","5,142 - 6,078",5142.0,6078.0,5610.0,111%,"Rs.7,268 - 8,882","7,268 - 8,882",7268.0,8882.0,8075.0,160%,NaN,NaN,NaN,NaN,NaN,NaN


### Assigning Pincodes to each locality

In [13]:
url="https://nominatim.openstreetmap.org/search/Andheri%20Mumbai?format=json&addressdetails=1&limit=1&polygon_svg=1"
result=requests.get(url).json()
result[0]['lat']

'19.1203713'

In [42]:
url="https://nominatim.openstreetmap.org/search/{}%20{}?format=json&addressdetails=1&limit=1&polygon_svg=1".format("4 Bungalows","Mumbai")
result=requests.get(url).json()
pp_1['Longitude']=None
pp_1.head()

From cffi callback <function _verify_callback at 0x000002C3EB01A9D8>:
Traceback (most recent call last):
  File "C:\Users\23165948\AppData\Local\Continuum\anaconda3\lib\site-packages\OpenSSL\SSL.py", line 306, in wrapper
    @wraps(callback)
KeyboardInterrupt


SSLError: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search/4%20Bungalows%20Mumbai?format=json&addressdetails=1&limit=1&polygon_svg=1 (Caused by SSLError(SSLError("bad handshake: Error([('SSL routines', 'tls_process_server_certificate', 'certificate verify failed')])")))

In [51]:

for index, row in pp_1.iterrows():

    locality=row['Locality Name']

#initialize your variable to None 


#loop until you get the coordinates

    #while(lat_lng_coords is None):
    url="https://nominatim.openstreetmap.org/search/{}%20{}?format=json&addressdetails=1&limit=1&polygon_svg=1".format(locality,"Mumbai")
    result=requests.get(url).json()
    if result!=[]:
        pp_1.loc[index,'Latitude']=result[0]['lat']
        pp_1.loc[index,'Longitude']=result[0]['lon']

    else:
        pp_1.loc[index,'Latitude']=None
        pp_1.loc[index,'Longitude']=None

        
#Assign Latitude and longitude row['Latitude']= lat_lng_coords[0] row['Longitude'] = lat_lng_coords[1]


In [52]:
pp_1.head()

,Locality Name,buy_rate,buy_rate_trim,buy_rate_lower,buy_rate_higher,buy_rate_avg,Buy Rates Q/Q,Rent_1B,Rent_1B_trim,Rent_1B_Lower,Rent_1B_Upper,Rent_1B_avg,Rent_1B_persqf,Rent_2B,Rent_2B_trim,Rent_2B_Lower,Rent_2B_Upper,Rent_2B_avg,Rent_2B_persqf,Rent_3B,Rent_3B_trim,Rent_3B_Lower,Rent_3B_Upper,Rent_3B_avg,Rent_3B_persqf,Latitude,Longitude
0,4 Bunglows,"Rs.18,828 - 20,825/sq. ft.","18,828 - 20,825",18828.0,20825.0,19826.5,-1.48%,"Rs.24,225 - 31,025","24,225 - 31,025",24225.0,31025.0,27625.0,139%,"Rs.43,350 - 48,450","43,350 - 48,450",43350.0,48450.0,45900.0,232%,"Rs.53,780 - 59,755","53,780 - 59,755",53780.0,59755.0,56767.5,286%,None,None
1,Aarey Milk Colony,"Rs.7,225 - 7,862/sq. ft.","7,225 - 7,862",7225.0,7862.0,7543.5,4.71%,"Rs.16,371 - 20,464","16,371 - 20,464",16371.0,20464.0,18417.5,244%,"Rs.21,216 - 23,338","21,216 - 23,338",21216.0,23338.0,22277.0,295%,"Rs.23,899 - 29,330","23,899 - 29,330",23899.0,29330.0,26614.5,353%,19.1561292,72.8707223
2,Abhyudaya Nagar,"Rs.26,605 - 28,858/sq. ft.","26,605 - 28,858",26605.0,28858.0,27731.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Rs.63,954 - 68,395","63,954 - 68,395",63954.0,68395.0,66174.5,239%,NaN,NaN,NaN,NaN,NaN,NaN,18.9904774,72.8440570930591
3,Adaigaon,"Rs.4,462 - 4,802/sq. ft.","4,462 - 4,802",4462.0,4802.0,4632.0,7.92%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
4,Adharwadi,"Rs.4,675 - 5,398/sq. ft.","4,675 - 5,398",4675.0,5398.0,5036.5,-7.26%,"Rs.5,142 - 6,078","5,142 - 6,078",5142.0,6078.0,5610.0,111%,"Rs.7,268 - 8,882","7,268 - 8,882",7268.0,8882.0,8075.0,160%,NaN,NaN,NaN,NaN,NaN,NaN,None,None


In [53]:
pp_1.to_csv("pp_latlong.csv")

In [57]:
city_data=pd.read_csv("https://raw.githubusercontent.com/dibyendutapadar/mumbai_city_analysis/master/pp_latlong.csv")
city_data.drop([''])

,Unnamed: 0,Locality Name,buy_rate,buy_rate_trim,buy_rate_lower,buy_rate_higher,buy_rate_avg,Buy Rates Q/Q,Rent_1B,Rent_1B_trim,Rent_1B_Lower,Rent_1B_Upper,Rent_1B_avg,Rent_1B_persqf,Rent_2B,Rent_2B_trim,Rent_2B_Lower,Rent_2B_Upper,Rent_2B_avg,Rent_2B_persqf,Rent_3B,Rent_3B_trim,Rent_3B_Lower,Rent_3B_Upper,Rent_3B_avg,Rent_3B_persqf,Latitude,Longitude
0,0,4 Bunglows,"Rs.18,828 - 20,825/sq. ft.","18,828 - 20,825",18828.0,20825.0,19826.5,-1.48%,"Rs.24,225 - 31,025","24,225 - 31,025",24225.0,31025.0,27625.0,139%,"Rs.43,350 - 48,450","43,350 - 48,450",43350.0,48450.0,45900.0,232%,"Rs.53,780 - 59,755","53,780 - 59,755",53780.0,59755.0,56767.5,286%,NaN,NaN
1,1,Aarey Milk Colony,"Rs.7,225 - 7,862/sq. ft.","7,225 - 7,862",7225.0,7862.0,7543.5,4.71%,"Rs.16,371 - 20,464","16,371 - 20,464",16371.0,20464.0,18417.5,244%,"Rs.21,216 - 23,338","21,216 - 23,338",21216.0,23338.0,22277.0,295%,"Rs.23,899 - 29,330","23,899 - 29,330",23899.0,29330.0,26614.5,353%,19.156129,72.870722
2,2,Abhyudaya Nagar,"Rs.26,605 - 28,858/sq. ft.","26,605 - 28,858",26605.0,28858.0,27731.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Rs.63,954 - 68,395","63,954 - 68,395",63954.0,68395.0,66174.5,239%,NaN,NaN,NaN,NaN,NaN,NaN,18.990477,72.844057
3,3,Adaigaon,"Rs.4,462 - 4,802/sq. ft.","4,462 - 4,802",4462.0,4802.0,4632.0,7.92%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Adharwadi,"Rs.4,675 - 5,398/sq. ft.","4,675 - 5,398",4675.0,5398.0,5036.5,-7.26%,"Rs.5,142 - 6,078","5,142 - 6,078",5142.0,6078.0,5610.0,111%,"Rs.7,268 - 8,882","7,268 - 8,882",7268.0,8882.0,8075.0,160%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
